## step1:根据模板攻击\机器学习得到的结果构建方程组

- 模板攻击版本

In [112]:
import numpy as np
##  ====带模的多项式乘法====  ##
def poly_multi(X,m):
    n = X.shape[0]
    D1 = np.zeros((n,n),dtype = int)  
    D2 = np.zeros((n,n),dtype = int)
    
    for j in range(m+1):    ## 矩阵运算法    coeff_front
        D1[m][j] = X[m-j]

    X = X[::-1]
    for j in range(m+1,n):   ## 矩阵运算法    生成coeff_behind
        D2[m][j] = X[j-m-1]

    s1_coeff = (D1-D2)[m]
    return s1_coeff 
################################################
## =====构造结果矩阵（模板攻击）==== ##
##:: 根据模板攻击得到的结果矩阵构造方程组
## input: predict_file; metadat_file; z_file; save_file_url
## output: result_fun
def construct_result_fun(predict_file_url,metadata_file_url,z_file_url):
    predict_file = np.load(predict_file_url)
    result_fun = np.zeros([predict_file.shape[0],258],dtype = int)  ## 每一行对应一个方程，0-255是系数；256是z值；257表示第几组的方程
## 得到对应的方程：填充系数、Z值、第几组方程
    for i in range(predict_file.shape[0]):
        which_file = predict_file[i][0]
        which_trace = predict_file[i][1]
        which_poi = predict_file[i][2]
        attack_file = np.load(metadata_file_url + "\metadata_attacking_part{0}.npz".format(which_file))
        z_value = np.load(z_file_url + "\Z_unpack_part{0}.npz".format(which_file))
        poi = attack_file['index'][which_trace]
        Z=z_value['Z']
        C = z_value['C'][which_trace]  # 得到对应的C值
        
        result_fun[i,-1] = poi   # 第几组：0,1,2,3
        result_fun[i,256] = Z[which_trace,poi,which_poi]  # Z值
        result_fun[i,0:256] = poly_multi(C,which_poi)   # 系数

#     np.save(r"F:\paper_dilithium_0115\function_file\result_fun.npy",result_fun)  ## 保存结果方程
    return result_fun   
#########################################################################
## 把方程切分成4个
## input: 总的方程矩阵
## output: 4组方程
def partition_equation(result_fun,save_file_url):
    result_fun0 = np.zeros([1,258])
    result_fun1 = np.zeros([1,258])
    result_fun2 = np.zeros([1,258])
    result_fun3 = np.zeros([1,258])
    for i in range(result_fun.shape[0]):
        if result_fun[i,-1] == 0:
            result_fun0 = np.vstack((result_fun0,result_fun[i,:258]))
        elif result_fun[i,-1] == 1:
            result_fun1 = np.vstack((result_fun1,result_fun[i,:258]))
        elif result_fun[i,-1] == 2:
            result_fun2 = np.vstack((result_fun2,result_fun[i,:258]))
        elif result_fun[i,-1] == 3:
            result_fun3 = np.vstack((result_fun3,result_fun[i,:258]))

    result_fun_group0 = result_fun0[1:,:].astype(int)
    del result_fun0
    result_fun_group1 = result_fun1[1:,:].astype(int)
    del result_fun1
    result_fun_group2 = result_fun2[1:,:].astype(int)
    del result_fun2
    result_fun_group3 = result_fun3[1:,:].astype(int)
    del result_fun3        
    np.save(save_file_url + "\\result_fun_group0.npy",result_fun_group0)
    np.save(save_file_url + "\\result_fun_group1.npy",result_fun_group1)
    np.save(save_file_url + "\\result_fun_group2.npy",result_fun_group2)
    np.save(save_file_url + "\\result_fun_group3.npy",result_fun_group3)
############################################################################################################################
####                                 A     P     I                                                                      ####
############################################################################################################################
## 1.导入文件
metadata_file_url = r"F:\paper_dilithium_0115\buf_paper0118\metadata\metadata_files\attacking_metadata"  # + "\metadata_attacking_part{0}.npz"
z_file_url = r"F:\paper_dilithium_0115\buf_paper0118\metadata\Z_unpack\Attacking" # + "\Z_unpack_part{0}.npz"
predict_file_url = r"F:\paper_dilithium_0115\function_file\predict_file\model\0307\result_9000_z_27_diff_14.0.npy" ## 模板攻击得到的结果文件

## 2.构造方程
result_fun = construct_result_fun(predict_file_url,metadata_file_url,z_file_url)  # 调用函数,构造方程

## 3.切割方程
save_file_url = r"F:\paper_dilithium_0115\function_file"    # 输出的四组方程矩阵的存储位置
partition_equation(result_fun,save_file_url)

- 机器学习版本

In [40]:
import numpy as np
##  ====带模的多项式乘法====  ##
def poly_multi(X,m):
    n = X.shape[0]
    D1 = np.zeros((n,n),dtype = int)  
    D2 = np.zeros((n,n),dtype = int)
    
    for j in range(m+1):    ## 矩阵运算法    coeff_front
        D1[m][j] = X[m-j]

    X = X[::-1]
    for j in range(m+1,n):   ## 矩阵运算法    生成coeff_behind
        D2[m][j] = X[j-m-1]

    s1_coeff = (D1-D2)[m]
    return s1_coeff 
################################################
## =====构造结果矩阵（机器学习）==== ##
##:: 根据模板攻击得到的结果矩阵构造方程组
## input: predict_file; metadat_file; z_file; save_file_url
## output: result_fun
def construct_result_fun(predict_file_url,metadata_file_url,z_file_url):
    predict_file = np.load(predict_file_url)
    result_fun = np.zeros([predict_file.shape[0],258],dtype = int)  ## 每一行对应一个方程，0-255是系数；256是z值；257表示第几组的方程
## 得到对应的方程：填充系数、Z值、第几组方程
    for i in range(predict_file.shape[0]):
        which_file = predict_file[i][0]
        which_trace = predict_file[i][1]
        which_poi = predict_file[i][2]
        attack_file = np.load(metadata_file_url + "\metadata_attacking_part{0}.npz".format(which_file))
        z_value = np.load(z_file_url + "\Z_unpack_part{0}.npz".format(which_file))
        poi = attack_file['index'][which_trace]
        Z=z_value['Z']
        C = z_value['C'][which_trace]  # 得到对应的C值
        
        result_fun[i,-1] = poi   # 第几组：0,1,2,3
               #  Z值
        if len(bin((Z[which_trace,poi,which_poi]) & 0xff)) == 10: ## 如果z低八位的二进制数是 11** **** ，说明对应的cs1的值为负值
            result_fun[i,256] = ((Z[which_trace,poi,which_poi])&0xff)-256
        else:
            result_fun[i,256] = ((Z[which_trace,poi,which_poi]) & 0xff)
        result_fun[i,0:256] = poly_multi(C,which_poi)   # 系数

#     np.save(r"F:\paper_dilithium_0115\function_file\result_fun.npy",result_fun)  ## 保存结果方程
    return result_fun   
#########################################################################
## 把方程切分成4个
## input: 总的方程矩阵
## output: 4组方程
def partition_equation(result_fun,save_file_url):
    result_fun0 = np.zeros([1,258])
    result_fun1 = np.zeros([1,258])
    result_fun2 = np.zeros([1,258])
    result_fun3 = np.zeros([1,258])
    for i in range(result_fun.shape[0]):
        if result_fun[i,-1] == 0:
            result_fun0 = np.vstack((result_fun0,result_fun[i,:258]))
        elif result_fun[i,-1] == 1:
            result_fun1 = np.vstack((result_fun1,result_fun[i,:258]))
        elif result_fun[i,-1] == 2:
            result_fun2 = np.vstack((result_fun2,result_fun[i,:258]))
        elif result_fun[i,-1] == 3:
            result_fun3 = np.vstack((result_fun3,result_fun[i,:258]))

    result_fun_group0 = result_fun0[1:,:].astype(int)
    del result_fun0
    result_fun_group1 = result_fun1[1:,:].astype(int)
    del result_fun1
    result_fun_group2 = result_fun2[1:,:].astype(int)
    del result_fun2
    result_fun_group3 = result_fun3[1:,:].astype(int)
    del result_fun3        
    np.save(save_file_url + "\\result_fun_group0.npy",result_fun_group0)
    np.save(save_file_url + "\\result_fun_group1.npy",result_fun_group1)
    np.save(save_file_url + "\\result_fun_group2.npy",result_fun_group2)
    np.save(save_file_url + "\\result_fun_group3.npy",result_fun_group3)
############################################################################################################################
####                                 A     P     I                                                                      ####
############################################################################################################################
## 1.导入文件
metadata_file_url = r"F:\paper_dilithium_0115\buf_paper0118\metadata\metadata_files\attacking_metadata"  # + "\metadata_attacking_part{0}.npz"
z_file_url = r"F:\paper_dilithium_0115\buf_paper0118\metadata\Z_unpack\Attacking" # + "\Z_unpack_part{0}.npz"
predict_file_url = r"C:\Users\zhang\Desktop\data\result_4_1800_0.18_30.npy"  ## ML得到的结果文件

## 2.构造方程
result_fun = construct_result_fun(predict_file_url,metadata_file_url,z_file_url)  # 调用函数,构造方程

## 3.切割方程
save_file_url = r"F:\paper_dilithium_0115\test_fun_file\ML_result\fun_result"    # 输出的四组方程矩阵的存储位置
partition_equation(result_fun,save_file_url)

- 检查四组方程文件的正确方程数量（可选）

In [41]:
## 检验函数： 检查方程中正确的方程数量
import numpy as np
def check_fun(S1_file,check_fun_file_url):
    S1 = np.load(S1_file + "\s1_attack.npy")
    total_fun_num = 0
    total_real_fun_num = 0
    for i in range(4):
        fun_file = np.load(check_fun_file_url + "\\result_fun_group{0}.npy".format(i))
        total_fun_num += fun_file.shape[0]
        
        real_num = 0
        for j in range(fun_file.shape[0]):
            if np.sum(S1[0,i] * fun_file[j,:256]) == fun_file[j,256]:
                real_num += 1
        print("fun{0} accuracy：".format(i),real_num/fun_file.shape[0],"    (",real_num,",",fun_file.shape[0],")"," error: ",fun_file.shape[0]-real_num)
        total_real_fun_num += real_num
    print("function number:",total_fun_num,"  real function:",total_real_fun_num," accuracy:",total_real_fun_num/total_fun_num," error: ",total_fun_num-total_real_fun_num)
############################################################################################################################
####                                 A     P     I                                                                      ####
############################################################################################################################
S1_file = r"F:\paper_dilithium_0115\buf_paper0118\metadata\Z_unpack"
check_fun_file_url = r"F:\paper_dilithium_0115\test_fun_file\ML_result\fun_result"  # + "\result_fun_group{0}.npy"
check_fun(S1_file,check_fun_file_url)

fun0 accuracy： 0.9380281690140845     ( 333 , 355 )  error:  22
fun1 accuracy： 0.9313432835820895     ( 312 , 335 )  error:  23
fun2 accuracy： 0.9017341040462428     ( 312 , 346 )  error:  34
fun3 accuracy： 0.8888888888888888     ( 320 , 360 )  error:  40
function number: 1396   real function: 1277  accuracy: 0.914756446991404  error:  119


## Step 2:使用线性规划求解器求解四组超定方程，获得S1

In [42]:
import numpy as np
import random
from ortools.linear_solver import pywraplp

## --------------------------------------------------------------------------

## ====线性规划函数==== ##
## 求解四组方程，得到S1
## input: 四组方程文件路径fun_file_url； 预测的S1文件存储位置save_S1_url；  第几组fun_i

def milp(fun_file_url,save_S1_url,fun_i):
    const_fun = np.load(fun_file_url + "\\result_fun_group{0}.npy".format(fun_i))
    print("function file: ",const_fun.shape)
    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver("GLOP")
    ## 定义变量

    x = [solver.IntVar(-2, 2, f'x_{i}') for i in range(1, 257)]  

    y = [solver.IntVar(0, 1,f'y_{i}') for i in range(1,const_fun.shape[0]+1)]

    print("Number of variables =", solver.NumVariables())

    # 添加约束条件        # X = [-2 0 1 -1 1]
    constraint_coeff = const_fun
    constraints = []
    k = 100
    for i in range(constraint_coeff.shape[0]):
        linear_expression = - constraint_coeff[i,256]
        for j in range(256):
            linear_expression += constraint_coeff[i,j] * x[j] 
        constraints.append(linear_expression <= k * (1 - y[i]))
        constraints.append(linear_expression >= -k * (1 - y[i]))
    #     constraints.append(constraint_coeff[i,0] * x[0] + constraint_coeff[i,1] * x[1] + constraint_coeff[i,2] * x[2] + constraint_coeff[i,3] * x[3] + constraint_coeff[i,4] * x[4] - constraint_coeff[i,5] <= k * (1-y_{i}) )
    #     constraints.append(constraint_coeff[i,0] * x[0] + constraint_coeff[i,1] * x[1] + constraint_coeff[i,2] * x[2] + constraint_coeff[i,3] * x[3] + constraint_coeff[i,4] * x[4] - constraint_coeff[i,5] >= -k * (1-y_{i}) )

    for constraint in constraints:
        solver.Add(constraint)
    print("Number of constraints =", solver.NumConstraints())

    ## 设定目标函数
    solver.Maximize(sum(y))
    # 设置时间限制
    solver.set_time_limit(30000)
    # 启用输出
    solver.EnableOutput()
    # 设置相对MIP间隙
    # solver.parameters.SetDoubleParam('relative_mip_gap', 0.58)

    print(f"Solving with {solver.SolverVersion()}")
    status = solver.Solve()
    predict_S1 = np.zeros(256,dtype = float)
    if status == pywraplp.Solver.OPTIMAL:
        print("Solve successfully !!!")
        print("Solution:")
        print("Objective value =", solver.Objective().Value())
        i = 0
        for variable in x:
#             print(f"{variable.name()} =", variable.solution_value())
            predict_S1[i] = variable.solution_value()
            i += 1
        print("-------------------------------------------")
#         for variable in y:
#             print(f"{variable.name()} =", variable.solution_value())
    else:
        print("The problem does not have an optimal solution.")
    np.save(save_S1_url + "\predict_fun{0}.npy".format(fun_i),np.round(predict_S1))
############################################################################################################################
####                                 A     P     I                                                                      ####
############################################################################################################################
fun_file_url = r"F:\paper_dilithium_0115\test_fun_file\ML_result\fun_result"
save_S1_url = r"F:\paper_dilithium_0115\test_fun_file\ML_result\fun_result"
for i in range(4):
    milp(fun_file_url,save_S1_url,i)

function file:  (355, 258)
Number of variables = 611
Number of constraints = 710
Solving with Glop solver v9.8.3296
Solve successfully !!!
Solution:
Objective value = 351.0099999999999
-------------------------------------------
function file:  (335, 258)
Number of variables = 591
Number of constraints = 670
Solving with Glop solver v9.8.3296
Solve successfully !!!
Solution:
Objective value = 329.5399999999999
-------------------------------------------
function file:  (346, 258)
Number of variables = 602
Number of constraints = 692
Solving with Glop solver v9.8.3296
Solve successfully !!!
Solution:
Objective value = 339.7399999999999
-------------------------------------------
function file:  (360, 258)
Number of variables = 616
Number of constraints = 720
Solving with Glop solver v9.8.3296
Solve successfully !!!
Solution:
Objective value = 351.4999999999999
-------------------------------------------


- 检验求解得到的四组S1的正确率（可选）

In [43]:
## 检验S1： 检查预测的S1正确率
def check_S1(S1_file,predict_S1_file_url):
    S1 = np.load(S1_file + "\s1_attack.npy")
    
    total_real_s1_num = 0
    for i in range(4):
        pre_s1 = np.load(predict_S1_file_url + "\\predict_fun{0}.npy".format(i))
        real_S1 = S1[0,i]
        
        real_num = np.sum(pre_s1 == real_S1)
        
        print("predict S1_{0} accuracy：".format(i),real_num/256,"    real:",real_num)
        total_real_s1_num += real_num
    print("Total S1 accuracy:",total_real_s1_num/1024,"  (",total_real_s1_num,",",1024,")")
############################################################################################################################
####                                 A     P     I                                                                      ####
############################################################################################################################
S1_file = r"F:\paper_dilithium_0115\buf_paper0118\metadata\Z_unpack"
predict_S1_file_url = r"F:\paper_dilithium_0115\test_fun_file\ML_result\fun_result"  # + "\result_fun_group{0}.npy"
check_S1(S1_file,predict_S1_file_url)

predict S1_0 accuracy： 1.0     real: 256
predict S1_1 accuracy： 1.0     real: 256
predict S1_2 accuracy： 1.0     real: 256
predict S1_3 accuracy： 1.0     real: 256
Total S1 accuracy: 1.0   ( 1024 , 1024 )


In [102]:
## ===自检程序=== ##
## 检查得到的解能和多少方程匹配
def check_self(file_url):
    for i in range(4):
        predict_s1 = np.load(file_url + "\\predict_fun{0}.npy".format(i))
        function = np.load(file_url + "\\result_fun_group{0}.npy".format(i))
        real_num = 0
        for j in range(function.shape[0]):
            if np.sum(function[j,:256]*predict_s1) == function[j,256]:
                real_num += 1
        print("fun{0} shape:".format(i),function.shape," real number:",real_num," error:",function.shape[0]-real_num)
#######################################################################################
file_url = r"F:\paper_dilithium_0115\function_file"
check_self(file_url)

fun0 shape: (299, 258)  real number: 283  error: 16
fun1 shape: (265, 258)  real number: 252  error: 13
fun2 shape: (315, 258)  real number: 300  error: 15
fun3 shape: (286, 258)  real number: 269  error: 17


In [54]:
a_file = np.load(r"F:\paper_dilithium_0115\function_file\predict_fun0.npy")
print(a_file.shape)
print(a_file)

(256,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [30]:
b = np.copy(a_file)
for i in range(b.shape[0]):
    b[i,0] += 4
print(b)
np.save(r"F:\paper_dilithium_0115\function_file\result_fun5.npy",b)

[[  4   1 108]
 [  4   1 162]
 [  4   1 220]
 ...
 [  6 496  57]
 [  6 497 226]
 [  6 498 100]]


In [6]:
import numpy as np
fun = np.load(r"F:\paper_dilithium_0115\function_file\result_fun_group3.npy")
print(fun.shape)

(268, 258)


In [3]:
for i in range(281):
    print(fun[i,-1])

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
